# 제 4 장 데이터 수집과 정리
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumMyeongjo')
rc('axes', unicode_minus=False)

___

## 4.1 __웹 스크래핑__

(1) 위키피디아 문서의 내용 가져오기

* 필요한 라이브러리 가져오기

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

* 방탄소년단에 대한 위키피디아 문서의 주소

In [ ]:
uri = 'https://ko.wikipedia.org/wiki/방탄소년단'

In [ ]:
uri_namu = 'https://namu.wiki/w/방탄소년단'

In [ ]:
resp = req.get(uri_namu)

In [ ]:
resp

* '방탄소년단' 위키피디아 페이지 가져오기 (BeautifulSoup 이용)

In [ ]:
# uri에 지정한 웹 페이지 가져오기
resp = req.get(uri)

In [ ]:
# 오류 발생 여부 확인
resp

In [ ]:
# HTML 문서 전체
wk_html = bs(resp.content,'html.parser')

* HTML 문서에서 스크래핑 하고자 하는 부분을 크롬 개발자 도구 등을 이용해 관련 정보를 가져옴

In [ ]:
# BeautifulSoup를 이용한 parsing
bts = wk_html.select_one('#mw-content-text > div.mw-content-ltr.mw-parser-output > p:nth-child(6)')

In [ ]:
bts

In [ ]:
bts.get_text()

(2) 나무 위키 문서의 내용 가져오기 (데이터베이스 덤프 가능)

참고 : https://huggingface.co/datasets/heegyu/namuwiki

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("heegyu/namuwiki") # 약 3GB

(3) 네이버 뉴스 검색 결과의 제목만 가져오기

In [ ]:
uri = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=방탄소년단'

In [ ]:
resp = req.get(uri)

In [ ]:
resp

In [ ]:
nv_html = bs(resp.content,'html.parser')

In [ ]:
bts_news = nv_html.select('.news_tit')

In [ ]:
bts_news

In [ ]:
for i in bts_news:
  print(i.get_text())

(4) 웹 사이트 테이블들을 데이터프레임으로 만들기

In [ ]:
code = '005930' # 삼성전자의 종목코드

In [ ]:
nf_uri = 'https://finance.naver.com/item/main.naver?code=' + code

In [ ]:
df_nf = pd.read_html(nf_uri,encoding='euc-kr')

In [ ]:
len(df_nf)

In [ ]:
df_nf[12]

(5) 우리 학교에서 시행 중인 교내장학금 제도

In [ ]:
uri = 'https://www.kyonggi.ac.kr/www/contents.do?key=5693'

In [ ]:
resp = req.get(uri)

In [ ]:
resp

In [ ]:
kgu_html = bs(resp.content,'html.parser')

In [ ]:
print("해당 페이지에 있는 모든 테이블의 클래스 :")
for table in kgu_html.find_all("table"):
    print(table.get("class"))

In [ ]:
table = kgu_html.select_one('table')

In [ ]:
table = kgu_html.find('table', class_='responsive')

In [ ]:
table

In [ ]:
mylist = []
for table_row in table.tbody.find_all("tr"):
    table_columns = table_row.find_all("td")

    if table_columns != []:
        x1 = table_columns[0].text
        x2 = table_columns[1].text
        x3 = table_columns[2].text
        x4 = table_columns[3].text

        mylist.append([x1,x2,x3,x4])

재학생장학금 = pd.DataFrame(
    mylist,
    columns=['장학금명','수혜대상 및 선발기준','장학금액 및 특전','수혜기간']
)

In [ ]:
table_row

In [ ]:
table_columns

In [ ]:
table_columns[1].text